In [24]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Advanced Tables
## Why are databases so complex?

* Data stored in a database may be split into multiple _tables_, each containing multiple _columns_. A column stores a single attribute of the data; a table stores a collection of related attributes.
* The database also keeps track of the relationships between different tables. 
* Databases are designed to minimize redundancy and maintain data integrity, particularly when data is added, changed, or deleted.

    1. Consistency when updating: no duplicate places for information https://en.wikipedia.org/wiki/Database_normalization
    2. Performance https://en.wikipedia.org/wiki/Star_schema
    
* Side note: you may also have to think about _isolation level_ when working with a database where someone may be updating data as you're trying to read it. The _isolation level_ determines the database read behavior in this situation. See https://en.wikipedia.org/wiki/Isolation_(database_systems).

## Working with multiple tables

* Two tables can be joined at a time. 'Join' is a binary operator. See https://en.wikipedia.org/wiki/Join_(SQL).
* Tables must have `key` values that can be matched. Usually one table has a `primary key` and the other table has a `foreign key`.


## Pandas

* Pandas allows "merge", "join", and "concatenate" operations. See http://pandas.pydata.org/pandas-docs/version/0.18.1/merging.html#merge-join-and-concatenate for additional reading.
* Pandas also allows reshaping and pivoting data tables, see http://pandas.pydata.org/pandas-docs/version/0.18.1/reshaping.html.


In this class, we will cover table joining, merging and concatenation. We will also go over using some of the time-series handling capabilities in Pandas.

In [25]:
import pandas as pd
import numpy as np

# Concatenating tables in Pandas

To introduce join operations, we will be working with the AdventureWorks dataset, a standard dataset from Microsoft SLQ Server for learing to work with databases. It contains data for the fictitious bicycle manufacturer (Adventure Works Cycles).

Let's starts by importing some tables from AdventureWorks in /home/data/AdventureWorks. These tables contain data on AdventureWorks employees, sales territories, customers, and orders placed by the customers.

In [101]:

    print(x)

(0,    EmployeeID  ManagerID  TerritoryID Title FirstName MiddleName LastName  \
0         259      250.0          NaN   NaN       Ben          T   Miller   

  Suffix JobTitle  NationalIDNumber      ...       SickLeaveHours  \
0    NaN    Buyer          20269531      ...                   47   

    PhoneNumber PhoneNumberType              EmailAddress   AddressLine1  \
0  151-555-0113            Work  ben0@adventure-works.com  101 Candy Rd.   

   AddressLine2     City StateProvinceName PostalCode    CountryName  
0           NaN  Redmond        Washington      98052  United States  

[1 rows x 26 columns])
(1,    EmployeeID  ManagerID  TerritoryID Title FirstName MiddleName LastName  \
1         278      274.0          6.0   NaN   Garrett          R   Vargas   

  Suffix              JobTitle  NationalIDNumber     ...     SickLeaveHours  \
1    NaN  Sales Representative         234474252     ...                 36   

    PhoneNumber PhoneNumberType               EmailAddress  \
1  

In [26]:
Employees = pd.read_excel('/home/data/AdventureWorks/Employees.xls')
Territory = pd.read_excel('/home/data/AdventureWorks/SalesTerritory.xls')
Customers = pd.read_excel('/home/data/AdventureWorks/Customers.xls')
Orders = pd.read_excel('/home/data/AdventureWorks/ItemsOrdered.xls')

Let's take a look at the data we'll be working with:

In [27]:
Employees.head()

,EmployeeID,ManagerID,TerritoryID,Title,FirstName,MiddleName,LastName,Suffix,JobTitle,NationalIDNumber,...,SickLeaveHours,PhoneNumber,PhoneNumberType,EmailAddress,AddressLine1,AddressLine2,City,StateProvinceName,PostalCode,CountryName
0,259,250.0,NaN,NaN,Ben,T,Miller,NaN,Buyer,20269531,...,47,151-555-0113,Work,ben0@adventure-works.com,101 Candy Rd.,NaN,Redmond,Washington,98052,United States
1,278,274.0,6.0,NaN,Garrett,R,Vargas,NaN,Sales Representative,234474252,...,36,922-555-0165,Work,garrett1@mapleleafmail.ca,10203 Acorn Avenue,NaN,Calgary,Alberta,T2P 2G8,Canada
2,204,26.0,NaN,NaN,Gabe,B,Mares,NaN,Production Technician - WC40,440379437,...,48,310-555-0117,Work,gabe0@adventure-works.com,1061 Buskrik Avenue,NaN,Edmonds,Washington,98020,United States
3,78,26.0,NaN,NaN,Reuben,H,D'sa,NaN,Production Supervisor - WC40,370989364,...,56,191-555-0112,Work,reuben0@adventure-works.com,1064 Slow Creek Road,NaN,Seattle,Washington,98104,United States
4,255,250.0,NaN,NaN,Gordon,L,Hee,NaN,Buyer,466142721,...,46,230-555-0144,Cell,gordon0@adventure-works.com,108 Lakeside Court,NaN,Bellevue,Washington,98004,United States


In [28]:
Territory.head()

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear
0,1,Northwest,US,North America,7887186.79,3298694.49
1,2,Northeast,US,North America,2402176.85,3607148.94
2,3,Central,US,North America,3072175.12,3205014.08
3,4,Southwest,US,North America,10510853.87,5366575.71
4,5,Southeast,US,North America,2538667.25,3925071.43


In [29]:
Customers.head()

,CustomerID,SalesTerritoryID,FirstName,LastName,City,StateName
0,10101,1,John,Gray,Lynden,Washington
1,10298,4,Leroy,Brown,Pinetop,Arizona
2,10299,1,Elroy,Keller,Snoqualmie,Washington
3,10315,3,Lisa,Jones,Oshkosh,Wisconsin
4,10325,1,Ginger,Schultz,Pocatello,Idaho


In [30]:
Orders.head()

,CustomerID,OrderDate,Item,Quantity,Price
0,10330,2004-06-30,Pogo stick,1,28.0
1,10101,2004-06-30,Raft,1,58.0
2,10298,2004-07-01,Skateboard,1,33.0
3,10101,2004-07-01,Life Vest,4,125.0
4,10299,2004-07-06,Parachute,1,1250.0


Let's construct a slightly artificial example. Suppose that AdventureWorks was formed by merging two companies, AdventuresUSA which operated in the US and AdventuresWorld, which operated in other countries. Now we want information on their combined sales territories. 

The Pandas "concat" function is good for stacking tables on top of each other. We will use it to combine the AdventuresUSA and AdventuresWorld territories data tables.

In [31]:
help(pd.concat)

Help on function concat in module pandas.tools.merge:

concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
    Concatenate pandas objects along a particular axis with optional set logic
    along the other axes. Can also add a layer of hierarchical indexing on the
    concatenation axis, which may be useful if the labels are the same (or
    overlapping) on the passed axis number
    
    Parameters
    ----------
    objs : a sequence or mapping of Series, DataFrame, or Panel objects
        If a dict is passed, the sorted keys will be used as the `keys`
        argument, unless it is passed, in which case the values will be
        selected (see below). Any None objects will be dropped silently unless
        they are all None in which case a ValueError will be raised
    axis : {0, 1, ...}, default 0
        The axis to concatenate along
    join : {'inner', 'outer'}, default 'outer'
        Ho

In [32]:
# constructing the territory tables... as noted, this is an artificial example
TerritoryUSA = Territory[Territory.CountryCode=='US']; TerritoryUSA['RepID'] = np.random.randint(1,1000,5)
TerritoryWorld = Territory[Territory.CountryCode!='US']

/usr/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [33]:
TerritoryUSA

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear,RepID
0,1,Northwest,US,North America,7887186.79,3298694.49,484
1,2,Northeast,US,North America,2402176.85,3607148.94,323
2,3,Central,US,North America,3072175.12,3205014.08,486
3,4,Southwest,US,North America,10510853.87,5366575.71,617
4,5,Southeast,US,North America,2538667.25,3925071.43,572


In [34]:
TerritoryWorld

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear
5,6,Canada,CA,North America,6771829.14,5693988.860
6,7,France,FR,Europe,4772398.31,2396539.760
7,8,Germany,DE,Europe,3805202.35,1307949.790
8,9,Australia,AU,Pacific,5977814.92,2278548.980
9,10,United Kingdom,GB,Europe,5012905.37,1635823.400
10,11,Brazil,BR,South America,0.00,261589.958
11,12,Mexico,MX,North America,0.00,0.000


In [35]:
# we'll concatenate the databases, but keep separate keys so that we can keep track of which entries came from AdventuresUSA and 
# which from AdventuresWorld.
# We'll use "join='inner'" to only keep colunms that are common to both tables; 
# that is, we will drop the no-longer needed RepID in AdventuresUSA. 
Territory2 = pd.concat([TerritoryUSA, TerritoryWorld], keys=['usa', 'world'], join='inner')

In [36]:
Territory2

TerritoryID            Name CountryCode         Region     SalesYTD  \
usa   0             1       Northwest          US  North America   7887186.79   
      1             2       Northeast          US  North America   2402176.85   
      2             3         Central          US  North America   3072175.12   
      3             4       Southwest          US  North America  10510853.87   
      4             5       Southeast          US  North America   2538667.25   
world 5             6          Canada          CA  North America   6771829.14   
      6             7          France          FR         Europe   4772398.31   
      7             8         Germany          DE         Europe   3805202.35   
      8             9       Australia          AU        Pacific   5977814.92   
      9            10  United Kingdom          GB         Europe   5012905.37   
      10           11          Brazil          BR  South America         0.00   
      11           12          Mexico          MX  North America         0.00   

          SalesLastYear  
usa   0     3298694.490  
      1     3607148.940  
      2     3205014.080  
      3     5366575.710  
      4     3925071.430  
world 5     5693988.860  
      6     2396539.760  
      7     1307949.790  
      8     2278548.980  
      9     1635823.400  
      10     261589.958  
      11          0.000

Pandas "append" behaves just like "concat" with axis=0 and join='outer' (i.e., keep all column names). Missing values are set to NaN. 

In [37]:
help(pd.DataFrame.append)

Help on function append in module pandas.core.frame:

append(self, other, ignore_index=False, verify_integrity=False)
    Append rows of `other` to the end of this frame, returning a new
    object. Columns not in this frame are added as new columns.
    
    Parameters
    ----------
    other : DataFrame or Series/dict-like object, or list of these
        The data to append.
    ignore_index : boolean, default False
        If True, do not use the index labels.
    verify_integrity : boolean, default False
        If True, raise ValueError on creating index with duplicates.
    
    Returns
    -------
    appended : DataFrame
    
    Notes
    -----
    If a list of dict/series is passed and the keys are all contained in
    the DataFrame's index, the order of the columns in the resulting
    DataFrame will be unchanged.
    
    See also
    --------
    pandas.concat : General function to concatenate DataFrame, Series
        or Panel objects
    
    Examples
    --------
    


In [38]:
Territory3 = TerritoryUSA.append(TerritoryWorld)

In [39]:
Territory3

,CountryCode,Name,Region,RepID,SalesLastYear,SalesYTD,TerritoryID
0,US,Northwest,North America,484.0,3298694.490,7887186.79,1
1,US,Northeast,North America,323.0,3607148.940,2402176.85,2
2,US,Central,North America,486.0,3205014.080,3072175.12,3
3,US,Southwest,North America,617.0,5366575.710,10510853.87,4
4,US,Southeast,North America,572.0,3925071.430,2538667.25,5
5,CA,Canada,North America,NaN,5693988.860,6771829.14,6
6,FR,France,Europe,NaN,2396539.760,4772398.31,7
7,DE,Germany,Europe,NaN,1307949.790,3805202.35,8
8,AU,Australia,Pacific,NaN,2278548.980,5977814.92,9
9,GB,United Kingdom,Europe,NaN,1635823.400,5012905.37,10


# Joining and merging tables in Pandas

Join and merge are powerful tools for working with multiple tables. We will use them to answer some questions about the
AdventureWorks dataset that you might encounter in real-life situations.

Join does fast table joining on a shared index. 
Merge does the same thing, but gives you the option to specify columns to join on. 
The idea of joining on a column will become clearer with some examples.

## Example 1. "I want a list of all employees, and if any are salespeople then show me the details about their sales territory"

From AdventureWorks, we have a table "Employees" that gives a lot of information about AdventureWorks employees, like 'EmployeeID', 'ManagerID', 'TerritoryID', 'Title', 'FirstName','MiddleName', 'LastName', 'Suffix', 'JobTitle', 'NationalIDNumber', 'BirthDate', 'MaritalStatus', 'Gender', 'HireDate', 'SalariedFlag', 'VacationHours', 'SickLeaveHours', 'PhoneNumber', 'PhoneNumberType', 'EmailAddress', 'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName', 'PostalCode', 'CountryName'. \

Since we're just being asked for a list of employees, we'll give the EmployeeID and their first, middle, and last names, and their role in the company (since additional information is requested for salespeople only). Then, for the salespeople, we must attach information about their sales territories, which is contained in the Territories table. 

Notice that the Employees table has a column 'TerritoryID', which corresponds to the primary key in the 'Territory' table (in 'Territory', each territory has a _unique_ 'TerritoryID'). We'll do a join on TerritoryID.

In [40]:
help(pd.merge)

Help on function merge in module pandas.tools.merge:

merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)
    Merge DataFrame objects by performing a database-style join operation by
    columns or indexes.
    
    If joining columns on columns, the DataFrame indexes *will be
    ignored*. Otherwise if joining indexes on indexes or indexes on a column or
    columns, the index will be passed on.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame
    how : {'left', 'right', 'outer', 'inner'}, default 'inner'
        * left: use only keys from left frame (SQL: left outer join)
        * right: use only keys from right frame (SQL: right outer join)
        * outer: use union of keys from both frames (SQL: full outer join)
        * inner: use intersection of keys from both frames (SQL: inner join)
    on : label or list
        Field names to jo

In [66]:
Ans = pd.merge(Employees.loc[:,["EmployeeID","FirstName","MiddleName","LastName","JobTitle","TerritoryID"]], 
               Territory, 
               how='left', on='TerritoryID')
Ans.head()

,EmployeeID,FirstName,MiddleName,LastName,JobTitle,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear
0,259,Ben,T,Miller,Buyer,NaN,NaN,NaN,NaN,NaN,NaN
1,278,Garrett,R,Vargas,Sales Representative,6.0,Canada,CA,North America,6771829.14,5693988.86
2,204,Gabe,B,Mares,Production Technician - WC40,NaN,NaN,NaN,NaN,NaN,NaN
3,78,Reuben,H,D'sa,Production Supervisor - WC40,NaN,NaN,NaN,NaN,NaN,NaN
4,255,Gordon,L,Hee,Buyer,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
X = Ans[["FirstName","MiddleName","LastName"]]

In [72]:
Ans['EmployeeName'] = X.apply(lambda x: x.LastName+", "+x.FirstName+" "+str(x.MiddleName), axis=1)

In [73]:
Ans

,EmployeeID,FirstName,MiddleName,LastName,JobTitle,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear,EmployeeName
0,259,Ben,T,Miller,Buyer,NaN,NaN,NaN,NaN,NaN,NaN,"Miller, Ben T"
1,278,Garrett,R,Vargas,Sales Representative,6.0,Canada,CA,North America,6771829.14,5693988.86,"Vargas, Garrett R"
2,204,Gabe,B,Mares,Production Technician - WC40,NaN,NaN,NaN,NaN,NaN,NaN,"Mares, Gabe B"
3,78,Reuben,H,D'sa,Production Supervisor - WC40,NaN,NaN,NaN,NaN,NaN,NaN,"D'sa, Reuben H"
4,255,Gordon,L,Hee,Buyer,NaN,NaN,NaN,NaN,NaN,NaN,"Hee, Gordon L"
5,66,Karan,R,Khanna,Production Technician - WC60,NaN,NaN,NaN,NaN,NaN,NaN,"Khanna, Karan R"
6,270,François,P,Ajenstat,Database Administrator,NaN,NaN,NaN,NaN,NaN,NaN,"Ajenstat, François P"
7,22,Sariya,E,Harnpadoungsataya,Marketing Specialist,NaN,NaN,NaN,NaN,NaN,NaN,"Harnpadoungsataya, Sariya E"
8,161,Kirk,J,Koenigsbauer,Production Technician - WC45,NaN,NaN,NaN,NaN,NaN,NaN,"Koenigsbauer, Kirk J"
9,124,Kim,T,Ralls,Stocker,NaN,NaN,NaN,NaN,NaN,NaN,"Ralls, Kim T"


In [42]:
# Overachiever answer:
Ans['EmployeeName'] = Ans[["FirstName","MiddleName","LastName"]].apply(lambda x: x.LastName+", "+x.FirstName+" "+str(x.MiddleName), axis=1)
Ans = Ans[['EmployeeName', 'EmployeeID', 'JobTitle', 'TerritoryID', 'Name', 'CountryCode', 'Region', 'SalesYTD', 'SalesLastYear']]
Ans

,EmployeeName,EmployeeID,JobTitle,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear
0,"Miller, Ben T",259,Buyer,NaN,NaN,NaN,NaN,NaN,NaN
1,"Vargas, Garrett R",278,Sales Representative,6.0,Canada,CA,North America,6771829.14,5693988.86
2,"Mares, Gabe B",204,Production Technician - WC40,NaN,NaN,NaN,NaN,NaN,NaN
3,"D'sa, Reuben H",78,Production Supervisor - WC40,NaN,NaN,NaN,NaN,NaN,NaN
4,"Hee, Gordon L",255,Buyer,NaN,NaN,NaN,NaN,NaN,NaN
5,"Khanna, Karan R",66,Production Technician - WC60,NaN,NaN,NaN,NaN,NaN,NaN
6,"Ajenstat, François P",270,Database Administrator,NaN,NaN,NaN,NaN,NaN,NaN
7,"Harnpadoungsataya, Sariya E",22,Marketing Specialist,NaN,NaN,NaN,NaN,NaN,NaN
8,"Koenigsbauer, Kirk J",161,Production Technician - WC45,NaN,NaN,NaN,NaN,NaN,NaN
9,"Ralls, Kim T",124,Stocker,NaN,NaN,NaN,NaN,NaN,NaN


### "For the list above, limit the results to just salespeople"

In [43]:
Ans2 = Ans[Ans.JobTitle=='Sales Representative']
Ans2

,EmployeeName,EmployeeID,JobTitle,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear
1,"Vargas, Garrett R",278,Sales Representative,6.0,Canada,CA,North America,6771829.14,5693988.86
54,"Campbell, David R",283,Sales Representative,1.0,Northwest,US,North America,7887186.79,3298694.49
63,"Mitchell, Linda C",276,Sales Representative,4.0,Southwest,US,North America,10510853.87,5366575.71
93,"Tsoflias, Lynn N",286,Sales Representative,9.0,Australia,AU,Pacific,5977814.92,2278548.98
114,"Mensa-Annan, Tete A",284,Sales Representative,1.0,Northwest,US,North America,7887186.79,3298694.49
163,"Ito, Shu K",281,Sales Representative,4.0,Southwest,US,North America,10510853.87,5366575.71
176,"Ansman-Wolfe, Pamela O",280,Sales Representative,1.0,Northwest,US,North America,7887186.79,3298694.49
230,"Carson, Jillian nan",277,Sales Representative,3.0,Central,US,North America,3072175.12,3205014.08
235,"Blythe, Michael G",275,Sales Representative,2.0,Northeast,US,North America,2402176.85,3607148.94
240,"Reiter, Tsvi Michael",279,Sales Representative,5.0,Southeast,US,North America,2538667.25,3925071.43


In [44]:
# Overachiever: What about *all* employees associated with sales?
Ans2 = Ans[Ans["JobTitle"].apply(lambda x: 'Sales' in x)]
Ans2

,EmployeeName,EmployeeID,JobTitle,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear
1,"Vargas, Garrett R",278,Sales Representative,6.0,Canada,CA,North America,6771829.14,5693988.86
54,"Campbell, David R",283,Sales Representative,1.0,Northwest,US,North America,7887186.79,3298694.49
59,"Jiang, Stephen Y",274,North American Sales Manager,NaN,NaN,NaN,NaN,NaN,NaN
63,"Mitchell, Linda C",276,Sales Representative,4.0,Southwest,US,North America,10510853.87,5366575.71
93,"Tsoflias, Lynn N",286,Sales Representative,9.0,Australia,AU,Pacific,5977814.92,2278548.98
114,"Mensa-Annan, Tete A",284,Sales Representative,1.0,Northwest,US,North America,7887186.79,3298694.49
135,"Alberts, Amy E",287,European Sales Manager,NaN,NaN,NaN,NaN,NaN,NaN
163,"Ito, Shu K",281,Sales Representative,4.0,Southwest,US,North America,10510853.87,5366575.71
176,"Ansman-Wolfe, Pamela O",280,Sales Representative,1.0,Northwest,US,North America,7887186.79,3298694.49
212,"Abbas, Syed E",285,Pacific Sales Manager,NaN,NaN,NaN,NaN,NaN,NaN


## "Give me a list of our customers, and also tell me which sales territory they fall in."

This looks like another question for "merge"! We have a list of customers with their addresses, and we have a list of territories, but they are in separate tables. 

Let's recover a list of customer names and IDs, together with corresponding sales territory names.

This time, we have to be careful, because "TerritoryID" in the Territory table matches "SalesTerritoryID" in the table Customers. So, we'll have to specify different columns names to merge on for the two tables.

In [45]:
Ans3 = pd.merge(Customers[["CustomerID","FirstName","LastName","SalesTerritoryID"]], 
                Territory[["TerritoryID","Name"]], 
                how='left', 
                left_on='SalesTerritoryID', right_on='TerritoryID', )
Ans3

,CustomerID,FirstName,LastName,SalesTerritoryID,TerritoryID,Name
0,10101,John,Gray,1,1,Northwest
1,10298,Leroy,Brown,4,4,Southwest
2,10299,Elroy,Keller,1,1,Northwest
3,10315,Lisa,Jones,3,3,Central
4,10325,Ginger,Schultz,1,1,Northwest
5,10329,Kelly,Mendoza,5,5,Southeast
6,10330,Shawn,Dalton,1,1,Northwest
7,10338,Michael,Howell,1,1,Northwest
8,10339,Anthony,Sanchez,4,4,Southwest
9,10408,Elroy,Cleaver,4,4,Southwest


## "Give me a list of all sales territories, also show what customers fall under them"

In [46]:
Ans = pd.merge(Territory, Customers, how="inner", left_on="TerritoryID", right_on="SalesTerritoryID")

In [47]:
Ans

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear,CustomerID,SalesTerritoryID,FirstName,LastName,City,StateName
0,1,Northwest,US,North America,7887186.79,3298694.49,10101,1,John,Gray,Lynden,Washington
1,1,Northwest,US,North America,7887186.79,3298694.49,10299,1,Elroy,Keller,Snoqualmie,Washington
2,1,Northwest,US,North America,7887186.79,3298694.49,10325,1,Ginger,Schultz,Pocatello,Idaho
3,1,Northwest,US,North America,7887186.79,3298694.49,10330,1,Shawn,Dalton,Cannon Beach,Oregon
4,1,Northwest,US,North America,7887186.79,3298694.49,10338,1,Michael,Howell,Tillamook,Oregon
5,3,Central,US,North America,3072175.12,3205014.08,10315,3,Lisa,Jones,Oshkosh,Wisconsin
6,3,Central,US,North America,3072175.12,3205014.08,10438,3,Kevin,Smith,Durango,Colorado
7,3,Central,US,North America,3072175.12,3205014.08,10439,3,Conrad,Giles,Telluride,Colorado
8,4,Southwest,US,North America,10510853.87,5366575.71,10298,4,Leroy,Brown,Pinetop,Arizona
9,4,Southwest,US,North America,10510853.87,5366575.71,10339,4,Anthony,Sanchez,Winslow,Arizona


## "Give me a list of the customers we have in North Carolina, and tell me how many there are."

In [109]:
# In-class exercise! :)

In [49]:
ListNC = Customers[Customers.StateName=='North Carolina']
ListNC

,CustomerID,SalesTerritoryID,FirstName,LastName,City,StateName
13,10429,5,Sarah,Graham,Greensboro,North Carolina


In [51]:
ListNC.CustomerID.count()

1

In [53]:
Customers = Customers.set_index("StateName")
Customers.loc['North Carolina']

CustomerID               10429
SalesTerritoryID             5
FirstName                Sarah
LastName                Graham
City                Greensboro
Name: North Carolina, dtype: object

## "For each of the items ordered, show the total price (sometimes they ordered more than 1 item)"

In [113]:
# We'll use the Orders table for this! In-class exercise :)

In [54]:
Orders["TotalItemPrice"] = Orders.Quantity * Orders.Price
Orders.head()

,CustomerID,OrderDate,Item,Quantity,Price,TotalItemPrice
0,10330,2004-06-30,Pogo stick,1,28.0,28.0
1,10101,2004-06-30,Raft,1,58.0,58.0
2,10298,2004-07-01,Skateboard,1,33.0,33.0
3,10101,2004-07-01,Life Vest,4,125.0,500.0
4,10299,2004-07-06,Parachute,1,1250.0,1250.0


In [55]:
Orders.groupby(["CustomerID","OrderDate"]).TotalItemPrice.sum()

CustomerID  OrderDate              
10101       2004-06-30 00:00:00.000      58.00
            2004-07-01 00:00:00.000     500.00
            2004-08-18 00:00:00.000      18.30
            2004-12-30 00:00:00.000      44.25
            2005-01-02 00:00:00.000      16.00
            2005-03-08 00:00:00.000     177.40
10298       2004-07-01 00:00:00.000      33.00
            2004-09-19 00:00:00.000      58.00
            2004-12-01 00:00:00.000      22.00
            2005-03-18 00:00:00.000      22.38
            2005-04-01 00:00:00.000      12.50
10299       2004-07-06 00:00:00.000    1250.00
            2005-01-18 00:00:00.000      38.00
10315       2005-02-02 00:00:00.000       8.00
10330       2004-06-30 00:00:00.000      28.00
            2005-01-01 00:00:00.000     112.00
            2005-12-31 23:59:59.997      16.75
10339       2004-07-27 00:00:00.000       4.50
10410       2004-10-28 00:00:00.000      89.22
            2005-01-30 00:00:00.000     192.50
10413       2005-01-19 0

## "Show a list of customers, and the total amount of money they have spent with AdventureWorks. I want the highest spenders to appear first!"

In [114]:
# In-class exercise! :)

In [56]:
Ans_1 = pd.merge(Customers[["CustomerID","FirstName","LastName"]], Orders[["CustomerID","TotalItemPrice"]], how="inner", on="CustomerID" )

In [57]:
Ans_1

,CustomerID,FirstName,LastName,TotalItemPrice
0,10101,John,Gray,58.00
1,10101,John,Gray,500.00
2,10101,John,Gray,18.30
3,10101,John,Gray,44.25
4,10101,John,Gray,16.00
5,10101,John,Gray,177.40
6,10298,Leroy,Brown,33.00
7,10298,Leroy,Brown,58.00
8,10298,Leroy,Brown,22.00
9,10298,Leroy,Brown,22.38


In [82]:
pd.DataFrame(Ans_1.groupby("CustomerID").TotalItemPrice.sum())

ValueError: Length mismatch: Expected axis has 11 elements, new values have 32 elements

In [91]:
Ans_1a = Ans_1[["FirstName","LastName","CustomerID"]].set_index("CustomerID")
Ans_1b = Ans_1a[Ans_1a.duplicated()==False]

In [92]:
Ans_2 = pd.DataFrame(Ans_1.groupby("CustomerID").TotalItemPrice.sum()).join(Ans_1b)

In [95]:
Ans_2.sort_values("TotalItemPrice", ascending=False)

,TotalItemPrice,FirstName,LastName
CustomerID,,,
10299,1288.00,Elroy,Keller
10449,970.79,Isabela,Moore
10101,813.95,John,Gray
10410,281.72,Mary Ann,Howell
10330,156.75,Shawn,Dalton
10298,147.88,Leroy,Brown
10439,139.00,Conrad,Giles
10413,128.00,Donald,Davids
10438,95.24,Kevin,Smith


In [84]:
Ans_2.sort_values(ascending=False)

TypeError: sort_values() missing 1 required positional argument: 'by'

In [75]:
Territory.apply(max, axis=0)

TerritoryID                  12
Name             United Kingdom
CountryCode                  US
Region            South America
SalesYTD            1.05109e+07
SalesLastYear       5.69399e+06
dtype: object

# Another side note:

In [10]:
help(pd.DataFrame.combine_first)

Help on function combine_first in module pandas.core.frame:

combine_first(self, other)
    Combine two DataFrame objects and default to non-null values in frame
    calling the method. Result index columns will be the union of the
    respective indexes and columns
    
    Parameters
    ----------
    other : DataFrame
    
    Examples
    --------
    a's values prioritized, use values from b to fill holes:
    
    >>> a.combine_first(b)
    
    
    Returns
    -------
    combined : DataFrame



In [11]:
help(pd.DataFrame.update)

Help on function update in module pandas.core.frame:

update(self, other, join='left', overwrite=True, filter_func=None, raise_conflict=False)
    Modify DataFrame in place using non-NA values from passed
    DataFrame. Aligns on indices
    
    Parameters
    ----------
    other : DataFrame, or object coercible into a DataFrame
    join : {'left'}, default 'left'
    overwrite : boolean, default True
        If True then overwrite values for common keys in the calling frame
    filter_func : callable(1d-array) -> 1d-array<boolean>, default None
        Can choose to replace values other than NA. Return True for values
        that should be updated
    raise_conflict : boolean
        If True, will raise an error if the DataFrame and other both
        contain data in the same place.



In [118]:

Customers

,CustomerID,SalesTerritoryID,FirstName,LastName,City,StateName
0,10101,1,John,Gray,Lynden,Washington
1,10298,4,Leroy,Brown,Pinetop,Arizona
2,10299,1,Elroy,Keller,Snoqualmie,Washington
3,10315,3,Lisa,Jones,Oshkosh,Wisconsin
4,10325,1,Ginger,Schultz,Pocatello,Idaho
5,10329,5,Kelly,Mendoza,Kailua,Hawaii
6,10330,1,Shawn,Dalton,Cannon Beach,Oregon
7,10338,1,Michael,Howell,Tillamook,Oregon
8,10339,4,Anthony,Sanchez,Winslow,Arizona
9,10408,4,Elroy,Cleaver,Globe,Arizona
